In [ ]:
import os
from pathlib import Path
import sys

CUR_DIR = Path.cwd()
ROOT_DIR = CUR_DIR.parent.parent
SRC_DIR = ROOT_DIR / 'src'

# Load python modules from the src directory
sys.path.append(str(SRC_DIR))
print(f"Added {SRC_DIR} to sys.path")

In [ ]:
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(SRC_DIR / '.env')

In [ ]:
import logging
from config import setup_logging

setup_logging(logging.INFO)

In [ ]:
from app import create_app_context

app_ctx = create_app_context()

Start a chat to turn on light "Lamp". When finished, you should see the following message in the output.

```
Assistant > The lamp is now turned on. If you need it set to a specific brightness or color, please let me know!
```

- https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/?pivots=programming-language-python
- https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/adding-openapi-plugins?pivots=programming-language-python

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.openapi_plugin import OpenAPIFunctionExecutionParameters
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory, ChatMessageContent, AuthorRole
from semantic_kernel.functions import KernelArguments

kernel = Kernel()

 # Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version="2024-10-21",
    ad_token_provider=app_ctx.cognitive_services_token_provider
)
kernel.add_service(chat_completion)

kernel.add_plugin_from_openapi(
    plugin_name="lights",
    openapi_document_path="http://localhost:3978/v1/swagger.json",
    execution_settings=OpenAPIFunctionExecutionParameters(
        # Inject headers include authorization and chat IDs
        # Check web server output to verify headers are sent correctly
        auth_callback=lambda: {'Authorization': "Bearer frank_token",
                               'Teams-Chat-Id': "19:example_chat_id@",
                               'Agent-Chat-id': "agent-asdfj-32456"},
        
        server_url_override="http://localhost:3978",

        # Determines whether payload parameter names are augmented with namespaces.
        # Namespaces prevent naming conflicts by adding the parent parameter name
        # as a prefix, separated by dots
        enable_payload_namespacing=True,
   ),
)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()
history.add_message(message=ChatMessageContent(role=AuthorRole.USER, content="Please turn on the lamp"))

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

# Print the results
print("Assistant > " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)